In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cp ./gdrive/My\ Drive/keras-yu/for443.zip ./for443.zip
!unzip ./for443.zip

In [8]:
cd for443

/content/for443


In [9]:
!pwd

/content/for443


In [0]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dropout
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
from keras.optimizers import SGD
from sklearn import metrics
from keras import backend as K

import os

In [0]:
TRAIN_DIR = './data/train/'
VAL_DIR = './data/validation/'
MODEL_OUT_DIR='weights/'
LOG_DIR = 'logs/'

EPOCHS=100
BATCH_SIZE = 16
CLASS_NUM = 2
THRESHOLD = 0.5

In [26]:
base_model = ResNet50(weights='imagenet', include_top=False,)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
#Dropout for Avoid Overfitting
x = Dropout(0.5)(x)
# and a logistic layer 
predictions  = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [30]:
# prepare data augmentation configuration
train_datagen = image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    vertical_flip=True,
    horizontal_flip=True,)

test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input,)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        VAL_DIR,
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        class_mode='binary')

Found 100 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [0]:
#Set callbacks
tb = TensorBoard(log_dir=LOG_DIR)
early_stopper = EarlyStopping(patience=5)
checkpointer = ModelCheckpoint(
    filepath=os.path.join(MODEL_OUT_DIR, 'modelWeights' + '-' + 'epoch:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5'),
    verbose=1,
    save_best_only=True)
csv_logger = CSVLogger(os.path.join(LOG_DIR, 'training_log-' + str(time.time()) + '.log'))

#precision
def Precision(y_true, y_pred):
    true_positives = K.sum(K.cast(K.greater(K.clip(y_true * y_pred, 0, 1), THRESHOLD), 'float32'))
    pred_positives = K.sum(K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), 'float32'))

    precision = true_positives / (pred_positives + K.epsilon())
    return precision

#recall
def Recall(y_true, y_pred):
    true_positives = K.sum(K.cast(K.greater(K.clip(y_true * y_pred, 0, 1), THRESHOLD), 'float32'))
    poss_positives = K.sum(K.cast(K.greater(K.clip(y_true, 0, 1), THRESHOLD), 'float32'))

    recall = true_positives / (poss_positives + K.epsilon())
    return recall
#Model Compiling
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9, nesterov=True), loss='binary_crossentropy',metrics=['accuracy',Precision,Recall])

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator.filenames) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[tb, early_stopper, csv_logger, checkpointer],
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames) // BATCH_SIZE)

Epoch 1/100
6/6 [==============================] - 177s 29s/step - loss: 0.8928 - acc: 0.3406 - Precision: 0.3337 - Recall: 0.3955 - val_loss: 0.8360 - val_acc: 0.3098 - val_Precision: 0.3400 - val_Recall: 0.4125

Epoch 00001: val_loss improved from inf to 0.83600, saving model to weights/modelWeights-epoch:001-val_acc:0.310.hdf5
Epoch 2/100
6/6 [==============================] - 182s 30s/step - loss: 0.8806 - acc: 0.4688 - Precision: 0.4985 - Recall: 0.5606 - val_loss: 0.8292 - val_acc: 0.3098 - val_Precision: 0.3312 - val_Recall: 0.3569

Epoch 00002: val_loss improved from 0.83600 to 0.82924, saving model to weights/modelWeights-epoch:002-val_acc:0.310.hdf5
Epoch 3/100
6/6 [==============================] - 168s 28s/step - loss: 0.8271 - acc: 0.4894 - Precision: 0.4165 - Recall: 0.4208 - val_loss: 0.7713 - val_acc: 0.4239 - val_Precision: 0.3447 - val_Recall: 0.3813

Epoch 00003: val_loss improved from 0.82924 to 0.77130, saving model to weights/modelWeights-epoch:003-val_acc:0.424.h

In [0]:
model.summary()

In [0]:
import pandas as pd
import numpy as np
import cv2
from keras import backend as K
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import load_model

K.set_learning_phase(1) #set learning phase



def Grad_Cam(input_model, x, layer_name):
    '''
    Args:
       input_model: モデルオブジェクト
       x: 画像(array)
       layer_name: 畳み込み層の名前

    Returns:
       jetcam: 影響の大きい箇所を色付けした画像(array)

    '''

    # 前処理
    X = np.expand_dims(x, axis=0)

    X = X.astype('float32')
    preprocessed_input = X


    # 予測クラスの算出

    predictions = model.predict(preprocessed_input)
    class_idx = np.argmax(predictions[0])
    class_output = model.output[:, class_idx]


    #  勾配を取得

    conv_output = model.get_layer(layer_name).output   # layer_nameのレイヤーのアウトプット
    grads = K.gradients(class_output, conv_output)[0]  # gradients(loss, variables) で、variablesのlossに関しての勾配を返す
    gradient_function = K.function([model.input], [conv_output, grads])  # model.inputを入力すると、conv_outputとgradsを出力する関数

    output, grads_val = gradient_function([preprocessed_input])
    output, grads_val = output[0], grads_val[0]

    # 重みを平均化して、レイヤーのアウトプットに乗じる
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)


    # 画像化してヒートマップにして合成

    cam = cv2.resize(cam, (224, 224), cv2.INTER_LINEAR) # 画像サイズは200で処理したので
    cam = np.maximum(cam, 0) 
    cam = cam / cam.max()

    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x / 2)   # もとの画像に合成

    return jetcam

In [0]:
x = img_to_array(load_img('data/validation/cat/cat.1.jpg', target_size=(224,224)))
array_to_img(x)

In [0]:
image = Grad_Cam(model, preprocess_input(x), 'activation_196') 
array_to_img(image)